# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from imputation import *
from inflation import *
from regression import *
from utils import *

In [3]:
estimation_method = 'cloud'
compute_percentile_threshold = 50  # select systems >= this percentile of training compute
variant = 'exclude-TPU'

if estimation_method == 'amortized':
    cost_estimation_function = estimate_amortized_hardware_costs
else:
    cost_estimation_function = estimate_costs

results_dir = f'results/{estimation_method}-{compute_percentile_threshold}th-{variant}/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_percentile_threshold=compute_percentile_threshold
)

In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(148, 5510, 142)

# Cost estimation

In [6]:
cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

==== System: Gemini Ultra ====
Skipping TPU Google TPU v4
==== System: Qwen-72B ====
Training time is required but no value found

==== System: Inflection-2 ====
Training time is required but no value found

==== System: Nemotron-3-8B ====
Trying NVIDIA A100 at 2023-08-28 00:00:00
Trying Amazon Web Services, Price per chip-hour (3-year CUD)
Found price: 1.45 at 2022-12-09 00:00:00
Difference between purchase time and price date: 262 days 00:00:00 

==== System: Yi-34B ====
Training time is required but no value found

==== System: Skywork-13B ====
Trying NVIDIA A800 at 2023-07-22 20:00:00
Trying Amazon Web Services, Price per chip-hour (3-year CUD)
Could not find price

Trying Microsoft Azure, Price per chip-hour (3-year CUD)
Could not find price

Trying Google Cloud, Price per chip-hour (3-year CUD)
Could not find price

Trying Amazon Web Services, Price per chip-hour (1-year CUD)
Could not find price

Trying Microsoft Azure, Price per chip-hour (1-year CUD)
Could not find price

Tryi

In [7]:
cost_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Organization categorization,Foundation model,Training compute lower bound,Training compute upper bound,Training chip-hours,Code accessibility,Dataset accessibility,Accessibility notes,Training time (chip hours),Cost
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Gemini Team,SOTA improvement,""" Evaluation on a broad range of benchmarks sh...",Hosted access (no API),https://storage.googleapis.com/deepmind-media/...,252.0,Gemini: A Family of Highly Capable Multimodal ...,...,Industry,NaN,NaN,NaN,132000000.0,NaN,NaN,NaN,132000000.0,NaN
46,Qwen-72B,Language,"Chat,Code generation","Jinze Bai, Shuai Bai, Yunfei Chu, Zeyu Cui, Ka...",SOTA improvement,"SOTA on several Chinese benchmarks, with highe...",Permissive license (depr.),https://huggingface.co/Qwen/Qwen-72B,NaN,NaN,...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,NaN,Significant use,Inflection-2 either already powers Pi or soon ...,API access,https://inflection.ai/inflection-2,NaN,Inflection-2: The Next Step Up,...,Industry,checked,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,Nemotron-3-8B,Language,"Chat,Language generation",NaN,SOTA improvement,"""The Nemotron-3-8B-QA model offers state-of-th...",Permissive license (depr.),https://developer.nvidia.com/blog/nvidia-ai-fo...,NaN,NVIDIA AI Foundation Models: Build Custom Ente...,...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,466944.0,677068.8
83,Yi-34B,Language,Chat,NaN,Significant use,2nd most popular model on HuggingFace: https:/...,Permissive license (depr.),https://arxiv.org/abs/2403.04652,NaN,NaN,...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,NaN,https://arxiv.org/abs/1609.08144,6105.0,Google's Neural Machine Translation System: Br...,...,Industry,NaN,NaN,NaN,414720.0,NaN,NaN,NaN,414720.0,178329.6
1131,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,NaN,https://www.nature.com/articles/nature16961,14733.0,Mastering the game of Go with deep neural netw...,...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1135,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,154061.0,Deep Residual Learning for Image Recognition,...,Industry,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2741.0,Deep Speech 2: End-to-End Speech Recognition i...,...,Industry,NaN,NaN,NaN,301.0,NaN,NaN,NaN,NaN,NaN


In [8]:
cost_df['Cost'].notna().sum()

38

Use the below to check data availability for specific systems

In [9]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['System'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [10]:
# TODO move to function

In [11]:
cost_df['Publication date']

39     2023-12-06
46     2023-11-30
54     2023-11-22
59     2023-11-15
83     2023-11-02
          ...    
1095   2016-09-26
1131   2016-01-27
1135   2015-12-10
1137   2015-12-08
1141   2015-10-01
Name: Publication date, Length: 148, dtype: datetime64[ns]

In [12]:
from_year_month = cost_df['Publication date'].apply(str)
cost_df['Publication date'] = from_year_month

In [13]:
cost_df['Publication date']

39      2023-12-06 00:00:00
46      2023-11-30 00:00:00
54      2023-11-22 00:00:00
59      2023-11-15 00:00:00
83      2023-11-02 00:00:00
               ...         
1095    2016-09-26 00:00:00
1131    2016-01-27 00:00:00
1135    2015-12-10 00:00:00
1137    2015-12-08 00:00:00
1141    2015-10-01 00:00:00
Name: Publication date, Length: 148, dtype: object

In [14]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2023-12-01')

In [15]:
cost_df['Cost (inflation-adjusted)'].dropna()

59      6.786538e+05
135     2.581632e+07
174     3.931897e+06
186     4.381249e+06
188     3.440490e+05
225     5.856264e+05
256     9.529677e+05
263     7.835203e+07
264     8.103978e+05
279     1.503763e+06
334     1.990912e+06
340     2.266037e+05
375     1.635573e+06
376     5.451909e+05
380     8.717377e+05
439     1.493975e+06
443     2.210876e+05
472     3.108410e+05
495     1.930600e+05
514     1.724109e+05
542     6.405653e+06
549     9.869716e+05
582     2.763895e+04
608     1.612272e+05
656     8.875309e+04
724     4.324883e+06
769     5.207392e+03
809     8.817783e+05
810     2.160704e+05
835     1.600176e+05
922     6.697565e+02
932     4.434402e+03
968     1.142688e+03
984     9.928160e+01
1041    3.344073e+04
1046    6.683843e+02
1094    2.028533e+04
1095    1.947392e+05
Name: Cost (inflation-adjusted), dtype: float64

In [16]:
cost_df['Cost (inflation-adjusted)'].notna().sum()

38

# Regression

In [17]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [18]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.523
Model:                            OLS   Adj. R-squared:                  0.510
Method:                 Least Squares   F-statistic:                     39.44
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           2.94e-07
Time:                        13:43:08   Log-Likelihood:                -48.791
No. Observations:                  38   AIC:                             101.6
Df Residuals:                      36   BIC:                             104.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -858.9443    137.628     -6.241      0.000   -1138.067    -579.822
x1             0.4276      0.068      6.280      0.000       0.290       0.566
==============================================================================
Omnibus:                        0.318   Durbin-Watson:                   2.042
Prob(Omnibus):                  0.853   Jarque-Bera (JB):                0.356
Skew:                           0.197   Prob(JB):                        0.837
Kurtosis:                       2.736   Cond. No.                     1.91e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.91e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
print_growth_rates(reg_results)

0.43 OOMs/year (95% CI: 0.29, 0.57)
2.7x/year (95% CI: 1.9x, 3.7x)
doubling time of 8 months (95% CI: 6, 12)


In [20]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [21]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,2.689117,0.448947,1.778611,3.599623,0.653514,4.724720,2015.00000,2015-01-01
1,2.732310,0.442446,1.834987,3.629632,0.702569,4.762050,2015.10101,2015-02-06
2,2.775503,0.435958,1.891340,3.659665,0.751545,4.799460,2015.20202,2015-03-15
3,2.818696,0.429481,1.947668,3.689723,0.800442,4.836949,2015.30303,2015-04-21
4,2.861888,0.423017,2.003970,3.719806,0.849258,4.874519,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,6.792442,0.271150,6.242524,7.342360,4.890582,8.694301,2024.59596,2024-08-06
96,6.835635,0.276977,6.273900,7.397369,4.930325,8.740944,2024.69697,2024-09-12
97,6.878828,0.282850,6.305181,7.452474,4.969972,8.787683,2024.79798,2024-10-19
98,6.922020,0.288768,6.336371,7.507670,5.009524,8.834517,2024.89899,2024-11-25


# Plots

In [22]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
if estimation_method == 'amortized':
    title_text = 'Amortized cost of hardware to train large ML systems'
else:
    title_text = 'Cost of cloud compute to train large ML systems'
fig.update_layout(title_text=title_text)

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=['2015-01-01', '2025-01-01'])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [23]:
label_systems = ['GNMT', 'Megatron-BERT', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini Ultra']

fig = px.scatter(
    cost_df.loc[cost_df['System'].isin(label_systems)],
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='System',
    log_y=True,
)

fig.add_scatter(
    x=cost_df['Publication date'],
    y=cost_df['Cost (inflation-adjusted)'],
    mode='markers',
    showlegend=False,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='95% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# legend on top
fig.update_layout(legend=dict(
    orientation='h',
    yanchor='top',
    y=-0.15,
    xanchor='center',
    x=0.5,
))

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2023 USD)')

# title
if estimation_method == 'amortized':
    title_text = 'Amortized cost of hardware to train large ML systems'
else:
    title_text = 'Cost of cloud compute to train large ML systems'
fig.update_layout(title_text=title_text)

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Export data

In [24]:
keep_cols = [
    'System',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,System,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
39,Gemini Ultra,Multimodal,"Language modelling,Visual question answering,C...",Hosted access (no API),Gemini: A Family of Highly Capable Multimodal ...,2023-12-06 00:00:00,Google DeepMind,NaN,5.000000e+25,NaN,...,Google TPU v4,Multinational,NaN,NaN,55000.0,NaN,NaN,NaN,NaN,NaN
46,Qwen-72B,Language,"Chat,Code generation",Permissive license (depr.),NaN,2023-11-30 00:00:00,Alibaba,7.200000e+10,1.300000e+24,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,Inflection-2,Language,Language modelling,API access,Inflection-2: The Next Step Up,2023-11-22 00:00:00,Inflection AI,NaN,1.001000e+25,NaN,...,NVIDIA H100 SXM5,United States of America,NaN,NaN,5000.0,NaN,NaN,NaN,NaN,NaN
59,Nemotron-3-8B,Language,"Chat,Language generation",Permissive license (depr.),NVIDIA AI Foundation Models: Build Custom Ente...,2023-11-15 00:00:00,NVIDIA,8.000000e+09,1.800000e+23,NaN,...,NVIDIA A100,United States of America,NaN,NaN,1024.0,0.34,NaN,NaN,677068.8,678653.827001
83,Yi-34B,Language,Chat,Permissive license (depr.),NaN,2023-11-02 00:00:00,01.AI,3.400000e+10,6.100000e+23,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,GNMT,Language,Translation,NaN,Google's Neural Machine Translation System: Br...,2016-09-26 00:00:00,Google,2.780000e+08,6.900000e+21,360000000.0,...,NVIDIA Tesla K80,United States of America,NaN,NaN,96.0,NaN,NaN,NaN,178329.6,194739.195303
1131,AlphaGo Lee,Games,Go,NaN,Mastering the game of Go with deep neural netw...,2016-01-27 00:00:00,DeepMind,NaN,1.900000e+21,29400000.0,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1135,ResNet-152 (ImageNet),Vision,Image classification,NaN,Deep Residual Learning for Image Recognition,2015-12-10 00:00:00,Microsoft,6.000000e+07,1.210000e+19,1280000.0,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,DeepSpeech2 (English),Speech,Speech recognition,NaN,Deep Speech 2: End-to-End Speech Recognition i...,2015-12-08 00:00:00,Baidu Research - Silicon Valley AI Lab,3.800000e+07,2.600000e+19,163339200.0,...,NVIDIA GTX Titan X,United States of America,NaN,NaN,NaN,0.45,NaN,NaN,NaN,NaN


In [25]:
cost_df[keep_cols].to_csv(results_dir + 'price dataset.csv', index=False)